In [1]:
# In this we want to generate noise plots from real-time recordings. There will be 4 plot sections. 1) The 
# regular array plot to show which electrodes are active in this buffer. 2) Time domain plots to show their 
# activity (the number will vary on the number of electrodes recorded for that buffer) 3) A heatmap plot that
# fills in the values and colors of the std of the electrode recordings 4) A histogram plot that fills in the 
# values of the electrodes over time.

In [177]:
## STEP 1: LOAD DEPENDENCIES, SET GLOBAL PARAMETERS, DEFINE FUNCTIONS

#General dependencies (some of these may not be used - check at the end)
import numpy as np
import scipy.io as sio
from os.path import dirname, join as pjoin
import time
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib tk
import pandas as pd
from matplotlib.animation import FuncAnimation
import glob
import os
import re
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Setting Params for plotting & troubleshoot
np.set_printoptions(threshold=np.inf)
plt.rcParams['animation.html']='jshtml'

# Reads for the largest file number of the specified data run
def func(p):
    return int(((p.split('.')[0]).split('_'))[-1])

def addlabels(x,y):
    for i in range(len(x)):
        ax4.annotate(y[i],(x[i],y[i]),ha='center',color='r',fontsize=9)

In [178]:
## STEP 2: INITIALIZE CHANGING VARIABLES

piece_no = 0
time_win = 20 #The amount of time (ms) you want for each analysis block
real_time_win = 300 #The amount of time (ms) you want analyzed total
max_samples = 200000 #Limit on the number of samples in the time domain plots to maintain real-time speeds
live = False
non_live_datarun = 24 # If looking at old data, enter in the datarun number
non_live_date_piece = '2021-08-13-0' # If looking at old data, enter in the date & piece
save = False

In [181]:
## STEP 3: INITIALIZE AUTOMATIC VARIABLES

# Note: This is different from real-time because we will be reading every file

# Path & Folder
path = r'/Shakedown/Users/vision/Desktop/Xilinx'

if live == True:
    
    date_piece = datetime.today().strftime('%Y-%m-%d') + '-' + str(piece_no)

    # Update the dataXXX folder to the latest one (assumes you want to plot noise tests happening now)
    if os.path.isdir(date_piece):
        dataruns = os.listdir(date_piece)
        last_datarun = 0
        for dir in dataruns:
            if dir.startswith('data'):
                val = int(re.findall('\d+',dir)[0])

                if val > last_datarun:
                    last_datarun = val

        run = last_datarun + 1

    else:
        run = 0
        
elif live != True:
    date_piece = non_live_date_piece
    run = non_live_datarun

# Initialize folder name and the prefix/file type for incoming buffer 
datarun = 'data'+str(run).zfill(3)
file_type = '/*mat'

# Initialize Figure Properties
plt.close('all')
fig = plt.figure(figsize=(14,12),facecolor='white',constrained_layout=False)
fig.show()
plt.style.use('fivethirtyeight')

grid = plt.GridSpec(2,2,width_ratios=[5,5],height_ratios=[5,5])
ax1 = plt.subplot(grid[0,0])
ax3 = plt.subplot(grid[1,0])
ax4 = plt.subplot(grid[1,1])
plt.subplots_adjust(left=0.25,right=0.94,bottom=0.05,top=0.95,wspace=0.25,hspace=0.2)

ax1.set_title('Whole Array Activity',{'fontsize':16})
ax3.set_title('Noise Matrix',{'fontsize':16})
ax4.set_title('Noise Histogram',{'fontsize':16})
textstr2 = 'Time Window = %.2f ms' %(time_win)
ax1.text(-0.8, 1.05, textstr2,transform=ax1.transAxes,fontsize=12,va = 'center')
ax1.set_xlim(left=-2,right=34)
ax1.set_ylim(bottom=-2,top=34)
textstr3 = 'GUI CONTROLS\n\n'
textstr4 = 'Pan/Zoom: Toggle [p], Click [l/r], Drag\nSave: [s]\nConstrain Pan/Zoom: Hold [x/y]\nPreserve Aspect Ratio: Hold [ctrl]\nReset: [r]'
ax1.text(-0.8,0.85,textstr3+textstr4,transform=ax1.transAxes,fontsize=9,va = 'center', ha = 'left', bbox={'facecolor':'white','edgecolor':'black','linewidth':1.5})

ax4.set_xlabel('Noise STD',{'fontsize':12})
ax4.set_ylabel('Number of Electrodes',{'fontsize':12})

# Initialize variables
max_cnt = real_time_win/0.05 #Saturation point for number of samples
max_dot = 600 #Saturation point for dot size
sample_win = int(time_win/0.05)

# Initialize Out of Loop Values
last_time = 0 #Allows us to increment the time domain plot if the same electrode is in 2 different buffers
cnt_time = 0 #Allows us to determine our max samples for a given electrode 
chan_num_pre = -1 #Setting the previous electrode in the count/time determination into an impossibility
buffer_num = 0 #Setting the first buffer incoming b/c we will be plotting all incoming buffers

noise_mean = np.zeros((32,32))
noise_std = np.zeros((32,32))
noise_cnt = np.zeros((32,32))

fig_rows_pre = 0
step = 1

## STEP 4: PRE-LOOP HOLDS (i.e. can run this before or after data is sent)

# If the file does not yet exist, stay here
while not(os.path.isdir(pjoin(date_piece, datarun))):
    continue

# If the file exists, but hasn't yet been filled, hold here
while True:
    data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)
    if len(data_dir)>0:
        break
        

In [182]:
# STEP 4: ALWAYS BE CHECKING FOR DATA

# Continuously Check for and Plot All Incoming Data (do we need this while loop if we are plotting one buffer at a time)
while True:
    data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)
    
    # Choose the file by the largest file number
    latest_file = max(data_dir, key=func)
    file_next = pjoin(date_piece,datarun,datarun+'_'+str(buffer_num)+'.mat')
    
    # If the next file to plot does not exist yet, hold here
    if func(file_next) > func(latest_file):
        continue
    elif file_next == latest_file:
        # In the off chance the file has been written, but not saved by 
        # the TCP socket yet, pause
        time.sleep(1)
    
    buffer_num += 1
    
    ########## TESTING MULTI ELEC ###########
    latest_file1 = 'Dummy_Data_MultiElec_Rand.mat'
    latest_file2 = 'Dummy_Data_MultiElec_Linear.mat'
    latest_file3 = 'Dummy_Data_CH586_TCP_Test.mat'
    
    if step == 1:
        file_next = latest_file1
        string = 'data_randnew'
        step = 2
    elif step == 2:
        file_next = latest_file2
        string = 'data_contnew'
        step = 3
    elif step == 3:
        file_next = latest_file3
        string = 'dummy_data_ch586_hard_gnd'
        step = 1
    
    ## STEP 5: PROCESS DATA 
    
    # Load the newest data (NOTE: Change to vect for full array test)
    mat_contents = sio.loadmat(file_next)
    data = mat_contents[string][0][:]
    
    # Convert the data
    data_real = np.zeros((32,32,len(data)-2))  #Initialize to max possible length. Note: Throw out first two values b/c garbo
    cnt_real = np.zeros((32,32,len(data)-2))
    #counter = np.zeros((1,1,len(data)-2)) - Uncomment once data w/ counter
    
    # Initialize Variables for the Loop
    cnt_pre = 0
    N = 0 # Sample times
    chan_index_pre = 1025 #Make sure it is not a true channel
    #counter_pre = 0 - Uncomment once data w/ counter
    
    for i in range(2,len(data)-1): 
        # Convert bit number into binary
        word = (np.binary_repr(data[i],32))
        
        # Break that binary into it's respective pieces and convert to bit number
        cnt = int(word[12:14],2)
        col = int(word[27:32],2)
        row = int(word[22:27],2)
        chan_index = row*32 + col
        #count_all = int(word[1:12],2) - Uncomment once data w/ counter
        
        # Only record the unique non-double count sample
        if(i==2 or (cnt_pre != cnt or chan_index != chan_index_pre)):
            
            # Sample time only changes when cnt changes
            if cnt != cnt_pre:
                N += 1
                
            # On the occurance the first cnt is not 0, make sure sample time is 0
            if i == 2:
                N = 0
                    
            # Uncomment this once data with counter comes in
            #if counter_pre != count_all:
                #count[0,0,counter_pre] = count_all
                
            # Update variables
            cnt_pre = cnt
            chan_index_pre = chan_index
            
            # Record pertinent data
            data_real[row][col][N] = int(word[14:22],2)
            cnt_real[row][col][N] = cnt
            
    # Truncate data to be the number of samples       
    num_samples = N
    data_real = data_real[:,:,:N]
    cnt_real = cnt_real[:,:,:N]
    #total_time = count_all*0.05 - Uncomment once data w/ counter
    
    ## STEP 6: PREP ARRAY FOR PLOTTING 
    
    # Initialize Array Indices
    base = np.arange(1,33)
    rows = [ele-1 for ele in base for i in range(32)]
    temp = [1 for i in range(32)]
    cols = [ele-1 * tele for tele in temp for ele in base]

    # Initialize Variables for the Loop (size and colors will be returned) 
    size = np.zeros((32,32,0)) # For each dot, size by # of samples
    num_sam = np.zeros((32,32,1)) #Temp variable to allow sample counting
    chan_sam = np.zeros((32,32,0)) #Number of samples for each channel in a given time window
    chan_avg = np.zeros((32,32,0)) #Avg amplitude of samples for each channel in a given time window
    colors = np.zeros((32,32,0)) # For each dot, color by avg amplitude
    avg_val = np.zeros((32,32,1)) #Temp variable for calculating average amplitude
    max_num_temp = np.zeros((2,1)) #Temp variable for sizing the dots in the array plot
    
    # Sample Counting
    num_sam[:,:,0] = np.count_nonzero(data_real,axis=2)
    incom_cnt = num_sam[:,:,0] 
    chan_sam = np.append(chan_sam,num_sam,axis=2) #num of samples per electrode
    fig_rows = np.count_nonzero(chan_sam)
    fig_elec = np.nonzero(chan_sam) #indices for electrodes with samples
#     if fig_rows > 8:
#         fig_rows = 8
#         fig_elec = np.argsort(chan_sam[:,:,0])[-8:,-8:]
    max_num_temp[0,0] = np.max(num_sam[:,:,0])
    
    # Sample Amplitude Averaging
    mask = data_real
    mask[data_real==0] = np.nan
    avg_val[:,:,0] = np.nanmean(mask,axis=2)
    avg_val = np.nan_to_num(avg_val,nan=0)
    incom_mean = avg_val[:,:,0]
    chan_avg = np.append(chan_avg,avg_val,axis=2)
    
    # Sample Amplitude Deviation
    incom_std = np.nanstd(mask,axis=2)
    incom_std = np.nan_to_num(incom_std,nan=0)
    
    # Features for Array Plot
    colors = np.append(chan_avg,avg_val,axis=2)
    scale = (max_dot-10)/max_num_temp[0,0]
    size = np.append(size, num_sam*scale+10,axis=2)
    
#     if max_num_temp[0,0] < max_cnt:
#         scale = (max_dot - 10) / (max_num_temp[0,0])
#         size = np.append(size, num_sam*scale + 10, axis = 2)
#     elif max_num_temp[0,0] >= max_cnt:
#         #For saturation, need to make > max_cnt = the same size
#         scale = (max_dot - 10) / (max_cnt)
#         size = np.append(size, num_sam*scale + 10, axis = 2)
    
    ## STEP 7: PREP HEATMAP & HISTOGRAM
    
    # Finalize Data for the Noise Matrix (updated with each buffer)
    pre_mean = noise_mean
    pre_std = noise_std
    pre_cnt = noise_cnt
    cnt_div = pre_cnt+incom_cnt
    cnt_div[cnt_div==0] = np.nan
    
    noise_mean = np.nan_to_num((pre_cnt*pre_mean + incom_cnt*incom_mean) / (cnt_div),nan=0)
    noise_std = np.sqrt(np.nan_to_num((pre_cnt*(pre_std**2+(pre_mean-noise_mean)**2)+incom_cnt*(incom_std**2+(incom_mean-noise_mean)**2))/(cnt_div),nan=0))
    noise_cnt = np.nan_to_num(pre_cnt + incom_cnt, nan=0)

    ## STEP 8: PREP TIME DOMAIN PLOTTING
    
    # Determine the Time of Each Sample (NOTE: Once counter is in place, this needs adjusted so any non-collision free samples appear void of values)
    total_time = num_samples*0.05 #Sampling rate 1/0.05 ms 
    times = np.linspace(0,total_time,num_samples)
    
    # I don't think we will worry about this in a noise test
#     times = times+last_time
#     last_time = times[len(times)-1]
#     cnt_time += len(times)
    

    if fig_rows > 8:
        fig_rows = 8
    ax2 = grid[0,1].subgridspec(fig_rows,1)
    
    # Remove Any Previous Time Domain Plots
    # Reset Figures if Plotting has Already Started
    if buffer_num-1 > 0:
        ax1.clear()
        for k in range(fig_rows_pre):
            axs_name[k].remove()
        ax3.clear()
        ax4.clear()
        cbar.remove()
        ax1.set_title('Whole Array Activity',{'fontsize':16})
        ax3.set_title('Noise Matrix',{'fontsize':16})
        ax4.set_title('Noise Histogram',{'fontsize':16})
        textstr2 = 'Time Window = %.2f ms' %(time_win)
        ax1.text(-0.8, 1.05, textstr2,transform=ax1.transAxes,fontsize=12,va = 'center')
        ax1.set_xlim(left=-2,right=34)
        ax1.set_ylim(bottom=-2,top=34)
        textstr3 = 'GUI CONTROLS\n\n'
        textstr4 = 'Pan/Zoom: Toggle [p], Click [l/r], Drag\nSave: [s]\nConstrain Pan/Zoom: Hold [x/y]\nPreserve Aspect Ratio: Hold [ctrl]\nReset: [r]'
        ax1.text(-0.8,0.85,textstr3+textstr4,transform=ax1.transAxes,fontsize=9,va = 'center', ha = 'left', bbox={'facecolor':'white','edgecolor':'black','linewidth':1.5})
        ax4.set_xlabel('Noise STD',{'fontsize':12})
        ax4.set_ylabel('Number of Electrodes',{'fontsize':12})
            
    fig_rows_pre = fig_rows
         
    # Need to plot each electrode with samples
    # Need to calc the std of each electrode with samples - heatmap
    # Histogram the std of each electrode with samples
    
#     if cnt_time >= max_samples:
#         ax2 = plt.subplot(grid[0,1])
#         ax2.clear()
#         ax2.set_title('Time Domain Plots')
#         ax2.text(-0.55,0,textstr3+textstr4,transform=ax2.transAxes,fontsize=9,bbox={'facecolor':'white','edgecolor':'black','linewidth':1.5})
#         ax2.set(xlabel = 'Time (ms)', ylabel = 'ADC')
        
#         cnt_time = 0
        
    ## STEP 8: PLOT THAT DATA
    
    # Initialize every time
    x1 = rows
    y1 = cols
    x2 = np.zeros((fig_rows,1,0))
    y2 = np.zeros((fig_rows,1,0))
    tempx = np.zeros((fig_rows,1,sample_win))
    tempy = np.zeros((fig_rows,1,sample_win))
    axs_name = []*fig_rows
    
    # Plot Array
    ax1.scatter(y1,x1,size[:,:,0],c=colors[:,:,0],cmap='jet')
    ax1.set_aspect('equal')
    
    # Plot Heatmap
    ax3.imshow(noise_std,cmap="plasma",origin='lower')
    ax3.set_aspect('equal')
    ax3.set_xticks(np.arange(data_real.shape[1]+1)-0.5,minor=True)
    ax3.set_yticks(np.arange(data_real.shape[0]+1)-0.5,minor=True)
    ax3.grid(False,which="major")
    ax3.grid(True,which="minor",axis='both',color="w",linestyle="-",linewidth=1)
    
#     for i in range(32):
#         for j in range(32):
#             text = ax3.text(j,i,noise_std[i,j],ha="center",va="center",color="b")
    
    
    # Plot Histogram
    cm = plt.cm.get_cmap('plasma')
    hist_cnt, hist_std = np.histogram(noise_std)
    hist_span = hist_std.max()-hist_std.min()
    C = [cm(((x-hist_std.min())/hist_span)) for x in hist_std]
    ax4.bar(hist_std[:-1],hist_cnt,color=C,edgecolor="black")
    asp = np.diff(ax4.get_xlim())[0]/np.diff(ax4.get_ylim())[0]
    ax4.set_aspect(asp)
    ax4.tick_params(axis='x',labelsize=10)
    ax4.tick_params(axis='y',labelsize=10)
    ax4.spines['left'].set_linewidth(2)
    ax4.spines['left'].set_color('black')
    ax4.spines['bottom'].set_linewidth(2)
    ax4.spines['bottom'].set_color('black')
    addlabels(hist_std[:-1],hist_cnt)
    axins = inset_axes(ax4, width="5%",height="90%",loc = 'center right',bbox_to_anchor=(0.1,0.,1,1),bbox_transform=ax4.transAxes)
    cbar = ax3.figure.colorbar(ax3.imshow(noise_std,cmap="plasma",origin='lower'),ax=ax4,cax=axins,pad=0.2)
    cbar.ax.tick_params(labelsize=9)
    cbar.ax.set_ylabel("Noise STD",rotation=-90,va="bottom",fontsize=11)
    
    # Initialize Time Domain Subplots
    for k in range(fig_rows):
        axs = fig.add_subplot(ax2[k,0])
        axs_name = np.append(axs_name,axs)
        text_elec = 'Elec # ' + str(fig_elec[0][k]*32+fig_elec[1][k])
        axs_name[k].annotate(text_elec,(0.01,0.75),xycoords='axes fraction',va='center',fontsize = 10,color='r')
            
    
    #Start with the last bit of information but plot all previous
    if total_time > real_time_win:
        i = num_samples - int(real_time_win/0.05)
        tempx2 = np.zeros((fig_rows,1,num_samples-int(real_time_win/0.05)-1))
        tempy2 = np.zeros((fig_rows,1,num_samples-int(real_time_win/0.05)-1))
        for k in range(fig_rows):
            tempx2[k,0,:] = times[0:num_samples-int(real_time_win/0.05)-1]
            tempy2[k,0,:] = data_real[fig_elec[0][k],fig_elec[1][k],0:num_samples-int(real_time_win/0.05)-1]
        x2 = np.append(x2,tempx2,axis=2)
        y2 = np.append(y2,tempy2,axis=2)
        for k in range(fig_rows):
            axs_name[k].plot(x2[k,0,:],y2[k,0,:],color='r',linewidth=2)
        fig.canvas.draw()
        fig.canvas.flush_events()
    elif total_time <= real_time_win:
        i = 0


    # Plot for the incoming data within real time window
    while num_samples > i:
        
        # For this time window, find the next values of x & y
        for k in range(fig_rows):
            tempx[k,0,:] = times[i:i+sample_win]
            tempy[k,0,:] = data_real[fig_elec[0][k],fig_elec[1][k],i:i+sample_win]

        x2 = np.append(x2,tempx,axis=2)
        y2 = np.append(y2,tempy,axis=2)
        
        for k in range(fig_rows):
            axs_name[k].plot(x2[k,0,:],y2[k,0,:],color='r',linewidth=2)
            axs_name[k].set_xlim(left=max(0,int(times[i]-5)),right=(int(times[i]))+time_win+1)
            #axs_name[k].set_ylim(bottom=0,top=250)
            axs_name[k].tick_params(axis='y',labelsize=(-2/7)*fig_rows+10+(2/7))
        
            if k == 0:
                axs_name[k].set_title('Time Domain Plots',{'fontsize':16})
                axs_name[k].tick_params(axis='x',labelsize=10)
            if k == fig_rows-1:
                axs_name[k].set_xlabel('Time (ms)', {'fontsize':12})
                axs_name[k].set_ylabel('ADC',{'fontsize':12})
            else:
                axs_name[k].set_xticklabels([])
            if fig_rows == 1:
                asp = np.diff(axs_name[0].get_xlim())[0]/np.diff(axs_name[0].get_ylim())[0]
                axs_name[0].set_aspect(asp)
                axs_name[k].spines['left'].set_linewidth(2)
                axs_name[k].spines['left'].set_color('black')
                axs_name[k].spines['bottom'].set_linewidth(2)
                axs_name[k].spines['bottom'].set_color('black')
            else:
                axs_name[k].spines['left'].set_linewidth(1)
                axs_name[k].spines['left'].set_color('black')
                axs_name[k].spines['bottom'].set_linewidth(1)
                axs_name[k].spines['bottom'].set_color('black')
                
   
            
        # This step is fast but most time consuming - may replace with fig.canvas.update()
        fig.canvas.draw()
        fig.canvas.flush_events()
    
        # Update the variables
        i += sample_win
            
        # Once at the end, go back to the beginning
        if i >= num_samples:
            break
        # If at the end of the buffer data, update temp variables to do the last loop
        if i + sample_win >= len(times):
            tempx = tempx[:,:,:len(times)-i]
            tempy = tempy[:,:,:len(times)-i]
            
            
        
        
# NOTE: Put in interrupt on updating plot to look for new data coming in
# Put in a Pause Button

<ipython-input-182-d91c66e3e32d>:127: RuntimeWarning: Mean of empty slice
  avg_val[:,:,0] = np.nanmean(mask,axis=2)
/Users/vision/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-182-d91c66e3e32d>:127: RuntimeWarning: Mean of empty slice
  avg_val[:,:,0] = np.nanmean(mask,axis=2)
/Users/vision/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-182-d91c66e3e32d>:127: RuntimeWarning: Mean of empty slice
  avg_val[:,:,0] = np.nanmean(mask,axis=2)
/Users/vision/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
<ipython-input-182-d91c66e3e32d>:127: RuntimeW

KeyboardInterrupt: 

In [ ]:
savefig('figname.png',facecolor=fig.get_facecolor(),transparent=True)

In [170]:
text_elec = 'Elec # ' + str(fig_elec[0][6]*32+fig_elec[1][6])
axs_name[6].annotate(text_elec,(0.01,0.75),xycoords='axes fraction',va='center',fontsize = 10,color='r')
#axs_name[7].text = (0.25,0.75,text_elec,{'fontsize':20,'color':'r','transform':axs_name[7].transAxes,})
fig.canvas.draw()
fig.canvas.flush_events()

In [161]:
print(fig_elec)
print(np.shape(axs_name))

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  7,  7,  

In [127]:
print(np.sort(chan_sam[:,:,0]))
print(np.shape(np.argsort(-chan_sam[:,:,0],axis=0)[:8]))
print()
print(np.argsort(-chan_sam[:,:,0],axis=0)[:8])
print(np.argsort(chan_sam[:,:,0],axis=1))


[[  0. 106. 107. 108. 111. 111. 112. 116. 116. 120. 120. 121. 123. 124.
  124. 124. 131. 132. 133. 135. 135. 135. 136. 136. 137. 138. 139. 141.
  141. 146. 147. 152.]
 [ 92. 103. 114. 115. 118. 119. 120. 120. 121. 122. 123. 124. 125. 125.
  127. 128. 129. 129. 131. 131. 132. 132. 132. 133. 135. 141. 142. 144.
  146. 148. 149. 155.]
 [106. 107. 109. 109. 114. 114. 114. 115. 119. 120. 120. 121. 121. 123.
  123. 125. 125. 125. 126. 127. 127. 128. 129. 129. 129. 132. 133. 133.
  135. 139. 140. 146.]
 [105. 106. 107. 110. 113. 116. 117. 117. 118. 119. 120. 120. 122. 122.
  123. 124. 126. 129. 129. 129. 130. 131. 131. 132. 133. 133. 134. 134.
  134. 137. 138. 146.]
 [101. 109. 111. 114. 114. 116. 119. 119. 119. 123. 125. 126. 127. 127.
  128. 129. 129. 130. 130. 132. 132. 132. 132. 134. 134. 136. 137. 137.
  137. 137. 143. 147.]
 [108. 109. 112. 114. 116. 116. 120. 121. 121. 122. 123. 124. 125. 125.
  126. 126. 127. 128. 129. 132. 132. 133. 136. 137. 138. 138. 138. 138.
  139. 141. 142. 143.

In [176]:
print(np.sort(chan_sam,axis=2))

[[[135.]
  [136.]
  [133.]
  [120.]
  [147.]
  [121.]
  [137.]
  [107.]
  [132.]
  [112.]
  [152.]
  [124.]
  [135.]
  [136.]
  [108.]
  [146.]
  [131.]
  [123.]
  [139.]
  [124.]
  [120.]
  [124.]
  [135.]
  [116.]
  [111.]
  [138.]
  [141.]
  [111.]
  [106.]
  [116.]
  [141.]
  [  0.]]

 [[125.]
  [115.]
  [129.]
  [122.]
  [132.]
  [144.]
  [149.]
  [146.]
  [129.]
  [119.]
  [142.]
  [133.]
  [118.]
  [123.]
  [127.]
  [131.]
  [ 92.]
  [132.]
  [132.]
  [135.]
  [114.]
  [120.]
  [124.]
  [141.]
  [155.]
  [125.]
  [131.]
  [121.]
  [120.]
  [103.]
  [128.]
  [148.]]

 [[114.]
  [119.]
  [109.]
  [128.]
  [139.]
  [129.]
  [140.]
  [126.]
  [123.]
  [129.]
  [125.]
  [114.]
  [107.]
  [125.]
  [115.]
  [121.]
  [120.]
  [120.]
  [121.]
  [123.]
  [146.]
  [109.]
  [114.]
  [127.]
  [106.]
  [132.]
  [129.]
  [133.]
  [127.]
  [125.]
  [133.]
  [135.]]

 [[130.]
  [117.]
  [137.]
  [117.]
  [116.]
  [124.]
  [126.]
  [131.]
  [118.]
  [134.]
  [129.]
  [122.]
  [129.]
  [110.]
  [1